In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.dates as mdates
from datetime import datetime 
from scipy.stats import lognorm, zscore, linregress
%matplotlib inline 
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import LabelEncoder  
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score


In [ ]:
fig_size = plt.rcParams["figure.figsize"]

# Set figure width to 12 and height to 9
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

In [2]:
#Import csv file
df=pd.read_csv('crap.csv', header=0)
pd.set_option('display.max_colwidth', 50)

### Data Analysis and Statistical Modeling Questions

#### 1. i. For the data set, identify high price outliers. Explain how you identified these outliers.

1.1 Outlier Detection
   Two cases come to mind:
       a) An input error by the contributor. This may an incorrect placement of a decimal point (i.e. quantity=1.00 -> quantity=100.).  The mistake can happen while inputing the price, quantity, size, units. Each of these contribute to the normalized price which will we use to find the outliers. 
       b) Given that contributors are financially compensated for the contributions, fraudulent submissions are innevitble. This often leads to a cat-and-mouse situation and the outlier algorithm needs to be updated
       
1.2 Not quite guassian Log-Normal plot(?):
    Giving that we do not expect prices to go below 0 and that decimal point mistakes can change the price values by order of magnitudes above and below the real price, a guassian distribution may not be suitable to model prices. On the otherhand, the log of the prices could follow a normal distributions so we use that.     
    
1.3 Method:
    After transforming the normalized price column to Log(normalized price), we define an outlier as a data point likes beyond 3 sigma of the mean. 

In [ ]:
#Empty dataframe to fill later
outliers = pd.DataFrame()

#Create dataframes who's content has ben grouped by Product
groupedDf = df.groupby(["p_item_product_lc"])


for prod_place, gp in groupedDf:
    #Add column of Log(normalized prices)
    gp.loc[:,'log_normalized_price'] = gp['normalized_price'].apply(lambda x: np.log(x))
    
    #Calculate z-score
    gp.loc[:,'zScore']= zscore(gp['log_normalized_price'])
    
    #store datapoints at least 1.96 sigma away
    z = gp[np.abs(gp["zScore"])>1.96]

    if not z.empty : 
        outliers = outliers.append(z)

In [6]:
#display all outliers by product
outliers.head()

,t_time,p_item_manufacturer_lc,p_item_brand_lc,p_item_sub_brand_lc,p_item_product_lc,p_item_description_lc,p_item_uuid,packaging,p_quantity,p_size,...,t_created,t_modified,t_uploaded_lc,g_timezone,g_source,thumbnail_0x0,thumbnail_300x300,u_uuid,log_normalized_price,zScore
94603,2015-01-08 10:48:49,NaN,NaN,NaN,Banana,NaN,spec:6877309b-f6c3-4073-853f-ff2652dcd1d8,per pcs.,1,1,...,2015-01-08 10:48:49,2015-01-12 20:38:06.577000,2015-01-08 10:48:49,Etc/GMT-0,offline,https://img.premise.com/0x0/a59542d8a005309636...,https://img.premise.com/300x300/a59542d8a00530...,e1f3044a-8703-4466-b043-1a383a518a86,0.300105,3.792949
94714,2015-01-26 18:37:14,NaN,NaN,NaN,Banana,NaN,spec:6877309b-f6c3-4073-853f-ff2652dcd1d8,per pcs.,3,1,...,2015-01-26 18:37:14,2015-01-27 17:59:08.533000,NaN,NaN,offline,https://img.premise.com/0x0/1bf3b3c9e514792852...,https://img.premise.com/300x300/1bf3b3c9e51479...,2132aed4-d164-464c-b5e5-85734f57f5ca,-0.405465,2.365355
94756,2015-01-30 16:04:40,NaN,NaN,NaN,Banana,NaN,spec:6877309b-f6c3-4073-853f-ff2652dcd1d8,per pcs.,5,5,...,2015-01-30 16:04:40,2015-01-31 01:24:06.443000,2015-01-30 16:04:40,Etc/GMT+0,offline,https://img.premise.com/0x0/eb893a58a232e3a0c4...,https://img.premise.com/300x300/eb893a58a232e3...,9a92e2f6-c752-4521-bf82-8ae2da985dc9,-3.218876,-3.327078
94768,2015-02-02 10:55:10,NaN,NaN,NaN,Banana,NaN,spec:6877309b-f6c3-4073-853f-ff2652dcd1d8,per pcs.,1,12,...,2015-02-02 10:55:10,2015-02-02 23:51:23.471000,NaN,NaN,offline,https://img.premise.com/0x0/ce5075c7fbb2f27ffb...,https://img.premise.com/300x300/ce5075c7fbb2f2...,bf494e87-79e5-40f9-bd99-41e449dc359b,-0.405465,2.365355
94800,2015-02-03 12:34:28,NaN,NaN,NaN,Banana,NaN,spec:6877309b-f6c3-4073-853f-ff2652dcd1d8,per pcs.,1,7,...,2015-02-03 12:34:28,2015-02-04 01:53:24.633000,2015-02-03 12:34:28,Etc/GMT+0,offline,https://img.premise.com/0x0/5523082f112fb5a948...,https://img.premise.com/300x300/5523082f112fb5...,555446cf-385b-4228-a124-5da8a6bbfbe9,-2.639057,-2.153919


I've noticed many Fruit Juice observations have a z score above ten.  Investingating a bit more:

In [ ]:
#Select Fruit Juice
fruitJuice = df[df["p_item_product_lc"]=="Fruit juice"]
#Plot all Fruit Juice normalized prices
fruitJuice.ix[:,"normalized_price"].plot(kind="hist",bins=100, logy=True, title="Normalized Fruit Juice Prices")

In [ ]:
fruitJuiceOutliers= outliers[outliers["p_item_product_lc"]=="Fruit juice"]
for contributor in fruitJuiceOutliers.groupby("u_uuid"):
    contributor[1].loc[:,"normalized_price"].plot(kind="hist",bins=100, logy=True, xlim=(0,.4),title="Outlier Fruit Juice prices by contributor")

Largest outliers in fruit juices seem to be from one contributor who has been submitting the same picture of a damaged can of fruit juice at various angles over the past 2 years. 

2.1
    i)  
        Contributor
        Location:
        Variations in set of products
    ii)

#### 2. i. For the data set, describe some variables that could be sources of sampling bias when estimating price trends in Ghana. Explain why each of these variables could cause sampling bias.

#### ii. Pick one bias (not u_uuid). Write some code that attempts to estimate the potential amount of sample bias caused by this variable, and describe your methodology.

In [ ]:
def plotTrend(frame=None, y="", product="",grp=None):
    grouped = frame.groupby(["p_item_product_lc"])
    prod = grouped.get_group("%s" % product) 
    #outlierProd = outliers[outliers["p_item_product_lc"]==product]
    #cleanedProd = prod.drop(outlierProd.index)
    prod.loc[:,"date"]= prod["t_time"].apply(lambda x : datetime.strptime(x.split(".")[0],"%Y-%m-%d %X").date())
    avgCC = prod.groupby(["date"]).mean()
    #display(avgCC)

    if grp is not None:
        for x in prod.groupby(grp):
            if len(x[1].groupby(["date"]))> 1.0:
                height = x[1][y].max() - x[1][y].min()
                print x[1].groupby(["date"]).mean().plot( y = "%s" % y, title="%s Bar Prices (%s)" % (product,x[0]), ylim =(prod[y].min()-5*height,prod[y].max()+5*height))
    else:
        height = prod[y].max() - prod[y].min()
        print prod.plot(x="date",y="%s" % y, title="%s Prices (%s)" % (product,y), ylim =(prod[y].min()-5*height,prod[y].max()+5*height))

    return 


We might want to look at the composition of the sample. i.e..sample by sample is a specific product over/under represented relative to other products.  For example, if normally, one expects on any given day that Coca-Cola bottles make up for 5% of submissions but yet for a specific day they made up 15% of the observations (maybe because contributors did not submit other products as much), then we can expect a bias in our sample.  

In [ ]:
#Add date column to dataframe
def addDate(dFrame):
    dFrame.loc[:,"date"]= dFrame["t_time"].apply(lambda x : datetime.strptime(x.split(".")[0],"%Y-%m-%d %X").date())
    return dFrame
#remove outliers from dataframe
df = addDate(df.drop(outliers.index))

Let's calculate the average daily price of each product for every purchase location for every city.

Keep in mind that that we need the additional requirement that all fields be filled.

In [ ]:
newDf = pd.DataFrame()
groups = ["p_item_product_lc","city","l_place_name","date"]
df = df.dropna(subset = groups)

#test=df[df["p_item_product_lc"]=="Banana"]
#display(test[test["date"]<datetime(2014, 11, 20).date()][groups])

prodCityPlaceDateGrouped = df.groupby(groups, as_index=True)
prodCityPlaceDate = prodCityPlaceDateGrouped["normalized_price"].agg([np.mean])
prodCityPlaceDate = prodCityPlaceDate.reset_index()
#test=prodCityPlaceDate[prodCityPlaceDate["p_item_product_lc"]=="Banana"]
#display(test[test["date"]<datetime(2014, 11, 20).date()])

#prodCityPlaceDate["normalized_price"]["mean"]
#prodDateGrouped = prodCityPlaceDate.groupby(["city","l_place_name"])
prodDateGrouped = prodCityPlaceDate.groupby(["p_item_product_lc","date"])
unbiased = prodDateGrouped["mean"].agg([np.mean])

biasedGrouped = df.groupby(["p_item_product_lc","date"], as_index=True)
biased = biasedGrouped["normalized_price"].agg([np.mean])
unbiased = unbiased.reset_index()
biased = biased.reset_index()
#unbiased.sort_values(by="date")
#display(unbiased.head())
#display(biased.head())
#plotTrend(frame=unbiased,y="mean",product="Banana")

def estimateBias(unbiased=None, biased=None):
    biases = [] 
    unbiased["bias_est"] = (unbiased["mean"]-biased["mean"])/ biased["mean"]
    for prod,prodDf in unbiased.groupby(["p_item_product_lc"]):
        days = [(e - min(prodDf["date"])).days for e in prodDf["date"]] 
        l = linregress(days,prodDf["bias_est"])
        biases.append(np.abs(l.intercept))
    return biases

maxB = max(estimateBias(unbiased, biased))
meanB = np.mean(estimateBias(unbiased, biased))
print "Average Bias in products due to imbalance sampling: %3.2f %%" % meanB



# Modelling Question


### 1. Create a model that predicts price from various metadata.

#### i. Explain how your model works, and why you chose it.

I have decided to use a random forest regression. A random forest averages over a number of decision trees (where each splitting branch is a feature of our model) trained on different sub-sets of the dataset. 

Decision Trees are able to pick out subtle features in the data (i.e. difference prices of onions bought at supermarkets vs open-markets) at the expense of potentially over-fitting the data. Creating a random selection of tress (a forest) helps to control over-fitting. Averaging over these threes also improves the predictive accuracy.

#### ii. Why did you use the metadata you used?

I used :
   * product: Not much to say there...can't compare apples and oranges...I need the product as a feature.
   * city: I expect variations in price in Metropolitan vs rural cities for many products. 
   * location: SuperMarkets and open markets will most likely offer differing product, but also at different prices.   
   * date: Using dates may allow our model to do some crude price forcasting (if time permits, I may try this.)

I did not use:
* package size, quantity, units: Since I'm predicting normalized price, those features are inheritly included.
* Product brand: I don't have a strong reason to not have included this, although I think the dimensionality of the dataset could increase too much for my model to handle, or it may expose it to over-fitting.  I'll come back to this and include if I have time.
   
#### iii. How can you be sure that you’re not over­fitting the model?

I split my dataset into 2/3 and 1/3 parts. The larger part is my training set while the latter is my testing set.
As epected the model gives a 100% accuracy on my training set ( a sign of potential over-fitting) but it is able to accuratly predict 92 percent of my testing set, which is pretty good. 


In [ ]:
#Sort by date 
prodCityPlaceDate = prodCityPlaceDate.sort_values(by="date")

In [ ]:
#Isolate predictor variables
X = prodCityPlaceDate.loc[:,groups]

#Transform categorical labels into integers
for group in groups:
        X.loc[:,group] = LabelEncoder().fit_transform(X.loc[:,group])

#Split dataset into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X,prodCityPlaceDate["mean"], test_size=0.33, random_state=42)

#instantiate model 
rfr = RandomForestRegressor(n_estimators=100, bootstrap=False)

#fit
rfr.fit(X_train,y_train)

In [ ]:
rfr.score(X_train,y_train)

In [ ]:
rfr.score(X_test,y_test)

In [7]:

#TrainDf = prodCityPlaceDate[prodCityPlaceDate["date"]<datetime(2015, 3, 17).date()]
#TestDf = prodCityPlaceDate[prodCityPlaceDate["date"]>=datetime(2015, 3, 17).date()]

scores = cross_val_score(rfr,X,prodCityPlaceDate["mean"], scoring='accuracy', cv=10)

TrainDf = sortedNewDF[sortedNewDF["date"]<datetime(2015, 3, 17).date()]
TestDf = sortedNewDF[sortedNewDF["date"]>=datetime(2015, 3, 17).date()]


for t in TrainDf.groupby(["p_item_product_lc"]):
    print t[0]

df

NameError: name 'rfr' is not defined